In [1]:
# langsmith 추적용 라이브러리

!pip install langchain-teddynote

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.8/124.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [2]:
# 외부 API 이용 시 필요 (ChatPerplextiy, ChatOpenAI)

!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15


In [3]:
# 오픈소스 임베딩 모델 이용 시 필요 (all-mpnet-based-v2)

!pip install langchain_huggingface

In [5]:
import os

from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "LangChain 실습"
# Langchain API key 입력
os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith_api_key')

In [6]:
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

from sklearn.metrics.pairwise import cosine_similarity

# 모델 저장
import joblib

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')



# Langsmith , Chat LLM 설정

In [7]:
from langchain_teddynote import logging

logging.langsmith("KOTRA_BL_data_matching")

LangSmith 추적을 시작합니다.
[프로젝트명]
KOTRA_BL_data_matching


In [17]:
from langchain_community.chat_models import ChatPerplexity
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

os.environ["PPLX_API_KEY"] = userdata.get('perplexity_api_key')
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')


In [21]:
# chat = ChatPerplexity(temperature=0,  model="llama-3.1-sonar-small-128k-chat")
chat = ChatOpenAI(temperature=0,  model="gpt-4o-mini")

<ipython-input-21-a37d487848de>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(temperature=0,  model="gpt-4o-mini")


# buyKOREA 설명 처리 프롬프트문

---------------------

+ 제품 설명 분석
+ 20개의 keyword를 부사, 형용사 없이 나열한 하나의 문장으로 출력
+ 제품의 용도를 잘 설명하는 20개의 한 단어 키워드 추출 (**중복 없이**)
+ 영어로 작성 (토큰 수 줄이기 + llm이 잘 인식하도록)
+ **키워드는 절이 아니라 단어여야 함 (다시 강조 - 절로 추출 시 필요없는 부사, 전치사 다수 포함)**  

In [38]:
prompt_buykorea = ChatPromptTemplate.from_messages([
    ("system", """Analyze product description.
    Make educated guesses based on product purpose/category.
    Extract 20 relevant, single-word keywords, no duplicates.
    Output as single sentence: 'keyword1, keyword2, keyword3,...'.
    Exclude adjectives/adverbs. For phrases, select key word only."""),
    ("human", "{input}")
])

chain_buykorea = prompt_buykorea | chat

In [39]:
paragraph = """Medium Buildable Coverage, Natural Matte Finish, Transformative Cream-To-Powder Texture
A tri-system blend of water, emollients and powder glides onto skin as a soft, creamy emulsion with a lightweight, transformative texture.
The innovative sweat- and humidity-resistant formula provides medium buildable coverage, allowing for true-to-skin shading from very light to very deep in a natural matte finish.
Light-diffusing properties soften lines to create a fresh, flawless texture for all skin types.
This formula also delivers immediate and long-term moisturization with 12-hour wear and won't cause acne.
Now available in 42 all-inclusive, colour-true shades
"""

response_buykorea = chain_buykorea.invoke({"input": paragraph})
print(response_buykorea.content)

coverage, finish, texture, blend, water, emollients, powder, skin, formula, shading, properties, lines, moisturization, wear, acne, shades, light, types, cream, matte.


# BL 데이터 이름 처리 프롬프트문

----------------------------

+ **제품 이름에 기반해, 제품 분류와 용도를 설명**
+ **BL 데이터에 존재하는 HSCODE 정보 입력**
+ 20개의 keyword를 부사, 형용사 없이 나열한 하나의 문장으로 출력
+ 제품의 용도를 잘 설명하는 20개의 한 단어 키워드 추출 (**중복 없이**)
+ 영어로 작성 (토큰 수 줄이기 + llm이 잘 인식하도록)
+ **키워드는 절이 아니라 단어여야 함 (다시 강조 - 절로 추출 시 필요없는 부사, 전치사 다수 포함)**  

In [33]:
hscode=330499

prompt_bldata = ChatPromptTemplate.from_messages([
    ("system", f"""Describe product category and purpose based on input.
     HS code: {hscode}.
   Extract 20 relevant, single-word keywords, no duplicates.
    Output only one sentence: 'keyword1, keyword2, keyword3,...'.
     Exclude adjectives/adverbs. For phrases, select key word only."""),
    ("human", "{input}")
])

chain_bldata = prompt_bldata | chat

# 같은 330499 내의 유사 품목 제품명
+ (순서대로 세팅 스프레이, 파운데이션, 컨실러)

# 키워드 추출 실험시 사용

P+P FIX+ COCONUT 100ML/3.4FLOZ - (100.00ml Each) - MXJJ010000

STUDIO TECH-NC25 10GM/.35OZ - (10.00 Each) - M1K2120000

STUDIO FINISH CONC-NC30 7GM/.24OZ - (7.00 g Each) - M450190000

In [34]:
name = """P+P FIX+ COCONUT 100ML/3.4FLOZ - (100.00ml Each) - MXJJ010000"""

response_bldata = chain_bldata.invoke({"input": name})
print(response_bldata.content)

coconut, oil, fragrance, cosmetic, personal, care, skin, hair, moisturizer, treatment, essence, extract, beauty, hygiene, product, liquid, bottle, health, wellness, aroma.


# 임베딩

In [14]:
from langchain_huggingface import HuggingFaceEmbeddings

# 오픈소스 임베딩 모델 다운로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [40]:
# 임베딩할 텍스트
text_buykorea = response_buykorea.content
text_bldata = response_bldata.content

# 텍스트 임베딩
embedding_buykorea = embeddings.embed_query(text_buykorea)
embedding_bldata = embeddings.embed_query(text_bldata)

# 결과 출력
print(f"전처리한 바이코리아 데이터 : {text_buykorea}")
print(f"전처리한 BL 데이터 : {text_bldata}")

print(f"코사인 유사도 : {cosine_similarity(np.array(embedding_buykorea).reshape(1,-1), np.array(embedding_bldata).reshape(1,-1))}")

전처리한 바이코리아 데이터 : coverage, finish, texture, blend, water, emollients, powder, skin, formula, shading, properties, lines, moisturization, wear, acne, shades, light, types, cream, matte.
전처리한 BL 데이터 : coconut, oil, fragrance, cosmetic, personal, care, skin, hair, moisturizer, treatment, essence, extract, beauty, hygiene, product, liquid, bottle, health, wellness, aroma.
코사인 유사도 : [[0.75259705]]


# LLM으로 제품 설명/이름 전처리하지 않을 시

In [41]:
# 텍스트 임베딩
embedding_buykorea_raw = embeddings.embed_query(paragraph)
embedding_bldata_raw = embeddings.embed_query(name)

# 결과 출력
print(f"원본 바이코리아 데이터 : {paragraph}")
print(f"원본 BL 데이터 : {name}")

print(f"코사인 유사도 : {cosine_similarity(np.array(embedding_buykorea_raw).reshape(1,-1), np.array(embedding_bldata_raw).reshape(1,-1))}")

원본 바이코리아 데이터 : Medium Buildable Coverage, Natural Matte Finish, Transformative Cream-To-Powder Texture
A tri-system blend of water, emollients and powder glides onto skin as a soft, creamy emulsion with a lightweight, transformative texture.
The innovative sweat- and humidity-resistant formula provides medium buildable coverage, allowing for true-to-skin shading from very light to very deep in a natural matte finish.
Light-diffusing properties soften lines to create a fresh, flawless texture for all skin types.
This formula also delivers immediate and long-term moisturization with 12-hour wear and won't cause acne.
Now available in 42 all-inclusive, colour-true shades

원본 BL 데이터 : P+P FIX+ COCONUT 100ML/3.4FLOZ - (100.00ml Each) - MXJJ010000
코사인 유사도 : [[0.29397394]]
